# Automatically creating Jira tickets

### Necessary installs

In [ ]:
!pip install jira
!pip install pbr

### Setting up the connection to the API

In [ ]:
import os
from jira import JIRA
import warnings
import getpass
import traceback

warnings.filterwarnings('ignore')
options = {
    'server': 'http://' + os.environ['JIRA_IP'] + ':8080'
}

JIRA_USER='andy.challis'
JIRA_PASS=getpass.getpass()

jira = JIRA(options, basic_auth=(JIRA_USER, JIRA_PASS))

JIRA_PROJECT_KEY='PIPE'
JIRA_BOARD_NAME='PIPE board'

### Adding comments to an issue

In [ ]:
issue = jira.issue('PIPE-1')

In [ ]:
jira.add_comment(issue, "I've done it!")

### Creating an issue

In [ ]:
issue_dict = {
    'project': {'key': JIRA_PROJECT_KEY},
    'summary': 'New issue from jira-python',
    'description': 'Look into this one',
    'issuetype': {'name': 'Task'},
}
new_issue = jira.create_issue(fields=issue_dict)

### User defined functions for pipeline logging

In [ ]:
def send_to_active_sprint(jira, issue_id):
    
    for board in jira.boards():
        if board.name == JIRA_BOARD_NAME:
            our_board_id = board.id
    
    for sprint in jira.sprints(our_board_id):
        if jira.sprint_info(our_board_id, sprint.id).get('state') == 'ACTIVE':
            current_sprint = sprint.id
            
    jira.add_issues_to_sprint(current_sprint, [issue_id])
        

def pipeline_complete(jira):
    
    issue_dict = {
        'project': {'key': JIRA_PROJECT_KEY},
        'summary': 'Pipeline complete',
        'description': 'Data needs moving from Location A  Environment X to location B on Environment Y',
        'issuetype': {'name': 'Task'},
    }
    issue = jira.create_issue(fields=issue_dict)
    
    send_to_active_sprint(jira, issue.key)
        

def pipeline_error(jira, error):
    
    issue_dict = {
        'project': {'key': JIRA_PROJECT_KEY},
        'summary': 'Pipeline Failed!',
        'description': 'The pipeline needs to be re run as soon as possible! \n\n The Exception was ' + error,
        'issuetype': {'name': 'Task'},
    }
    issue=jira.create_issue(fields=issue_dict)
    
    send_to_active_sprint(jira, issue.key)

        
def jira_full_pipeline(models, jira):
    
    try:
        for model in machine_learning_models:
            print("I like " + model + " models")
        
        pipeline_complete(jira)

    except:
        error=traceback.format_exc()
        pipeline_error(jira, error)
        raise SystemExit("Error! The Exception was " + error)

## Running a pipeline

In [ ]:
machine_learning_models = ["Regression", "Random Forest", "SVM"]
jira_full_pipeline(machine_learning_models, jira)